In [1]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

import funciones_auxiliares_hito_4 as aux4
from funciones_auxiliares_hito_2 import preprocesar_y_recodificar_enunciado_uno

# warnings.filterwarnings('ignore')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
df = pd.read_csv('income-db.csv')
df_final = preprocesar_y_recodificar_enunciado_uno(df)

# 1. Modelo completo

In [3]:
roc_acumulado = aux4.calcular_roc_cross_val(df_final, LogisticRegression(), 'income', 10)
roc_acumulado.mean()

0.9002500045669013

# 2. Modelo con variable p-value < 0.05

In [4]:
var_modelo_2 = ["age",
                "fnlwgt",
                "educational_num",
                "gender",
                "capital_gain",
                "capital_loss",
                "hours_per_week",
                "federal_gov",
                "self_employed",
                "divorced",
                "married",
                "separated",
                "widowed",
                "others",
                "white_collar",
                "Not_in_family",
                "Other_relative",
                "Own_child",
                "Unmarried",
                "Wife",
                "Asian_Pac_Islander",
                "White",
                "america",
                "asia",
                "europe"]

df_modelo_2 = df_final[var_modelo_2 + ['income']]


In [5]:
roc_modelo_2 = aux4.calcular_roc_cross_val(df_modelo_2, LogisticRegression(), 'income', 10)
roc_modelo_2.mean()

0.9003555292885993

# Modelo 3 - p value < 0.05 y coef >= 0.4

In [6]:
var_modelo_3 = ["gender",
                "federal_gov",
                "divorced",
                "married",
                "separated",
                "widowed",
                "others",
                "white_collar",
                "Not_in_family",
                "Other_relative",
                "Own_child",
                "Unmarried",
                "Wife",
                "Asian_Pac_Islander",
                "White",
                "america",
                "asia",
                "europe"]

df_modelo_3 = df_final[var_modelo_3 + ['income']]

In [7]:
roc_modelo_3 = aux4.calcular_roc_cross_val(df_modelo_3, LogisticRegression(), 'income', 10)
roc_modelo_3.mean()

0.8411547717143067